<img align="right" src="tf-small.png"/>

# Search

*Search* in Text-Fabric is a template based way of looking for structural patterns in your dataset.

It is inspired by the idea of
[topographic query](http://books.google.nl/books?id=9ggOBRz1dO4C),
as worked out in 
[MQL](https://shebanq.ancient-data.org/shebanq/static/docs/MQL-Query-Guide.pdf)
which has been implemented in 
[Emdros](http://emdros.org).
See also [pitfalls of MQL](https://etcbc.github.io/text-fabric-data/features/hebrew/etcbc4c/0_mql.html)

Within Text-Fabric we have the unique possibility to combine the ease of formulating search templates for
complicated syntactical patterns with the power of programmatically processing the results.

This notebook will show you how to get up and running.

See the notebook
[searchFromMQL](https://github.com/etcbc/text-fabric/blob/master/docs/searchFromMQL.ipynb)
for examples how MQL queries can be expressed in Text-Fabric search.

# Before we continue
Search is a big feature in Text-Fabric.
It is also a very recent addition.

##### Caution:
> There might be bugs.

Search is also costly.
Quite a bit of the implementation work has been dedicated to optimize performance.
But it is worth the price: search templates are powerful for a wide range of purposes.
I do not pretend, however, to have found optimal strategies for all 
possible search templates.

That being said, I think search might turn out helpful in many cases,
and I welcome your feedback.

*Dirk Roorda, 2016-12-23, updates 2017-01-04*

# Search command

It al starts by saying (just an example)

```
S.study('''
# here comes my search template:

clause
    p1:phrase det=und
        word sp=verb gn=f nu=pl ps=p3
        word sp=subs

    p2:phrase
    
  p1 < p2  
''')
```

See all ins and outs in the
[search template reference](https://github.com/ETCBC/text-fabric/wiki/Api#search-template-reference).

All search related things use the
[`S` api](https://github.com/ETCBC/text-fabric/wiki/Api#search).

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from tf.fabric import Fabric

In [3]:
ETCBC = 'hebrew/etcbc4c'
TF = Fabric( modules=ETCBC )

This is Text-Fabric 2.3.1
Api reference : https://github.com/ETCBC/text-fabric/wiki/Api
Tutorial      : https://github.com/ETCBC/text-fabric/blob/master/docs/tutorial.ipynb
Data sources  : https://github.com/ETCBC/text-fabric-data
Data docs     : https://etcbc.github.io/text-fabric-data
Shebanq docs  : https://shebanq.ancient-data.org/text
Slack team    : https://shebanq.slack.com/signup
Questions? Ask shebanq@ancient-data.org for an invite to Slack
108 features found and 0 ignored


Let us just *not* load any specific features.

In [4]:
api = TF.load('')
api.makeAvailableIn(globals())

  0.00s loading features ...
   |     0.00s Feature overview: 102 nodes; 5 edges; 1 configs; 7 computeds
  4.49s All features loaded/computed - for details use loadLog()


Here is a simple query to get started.
We are interested in two lexemes, but we would also like to fetch the nodes in their context.

##### Note
> This is not a very good use case, 
because in Text-Fabric it is easy to find context nodes around your nodes of interest.

In [5]:
query = '''
book
  chapter
    verse
      clause
        clause_atom
          phrase
            phrase_atom
              word lex=JC/|>JN/
'''

The next thing to do is to feed it to the search api, which will *study* it.
The syntax will be checked, features loaded, the search space will be set up, narrowed down, 
and the fetching of results will be prepared, but not yet executed.

In [6]:
S.study(query)

  0.00s Checking search template ...
  0.00s loading features ...
   |     0.21s B lex                  from /Users/dirk/github/text-fabric-data/hebrew/etcbc4c
  0.21s All additional features loaded - for details use loadLog()
  0.21s Setting up search space for 8 objects ...
  1.00s Constraining search space with 7 relations ...
  1.05s Setting up retrieval plan ...
  1.08s Ready to deliver results from 5870 nodes
Iterate over S.fetch() to get the results
See S.showPlan() to interpret the results


Before we rush to the results, lets have a look at the *plan*.

In [7]:
S.showPlan()

  3.88s The results are connected to the original search template as follows:
 0     
 1 R0  book
 2 R1    chapter
 3 R2      verse
 4 R3        clause
 5 R4          clause_atom
 6 R5            phrase
 7 R6              phrase_atom
 8 R7                word lex=JC/|>JN/
 9     


Here you see already what your results will look like.
Each result `r` is a *tuple* of nodes:
```
(R0, R1, R2, R3, R4, R5, R6, R7)
```
that instantiate the objects in your template.

## Excursion
In case you are curious, you can get details about the search space as well:

In [8]:
S.showPlan(details=True)

Search with 8 objects and 7 relations
Results are instantiations of the following objects:
node  0-book                              (    39   choices)
node  1-chapter                           (   416   choices)
node  2-verse                             (   799   choices)
node  3-clause                            (   922   choices)
node  4-clause_atom                       (   922   choices)
node  5-phrase                            (   923   choices)
node  6-phrase_atom                       (   923   choices)
node  7-word                              (   926   choices)
Instantiations are computed along the following relations:
node                      0-book          (    39   choices)
edge  0-book          [[  1-chapter       (    17.5 choices)
edge  1-chapter       [[  2-verse         (     2.0 choices)
edge  2-verse         [[  3-clause        (     1.1 choices)
edge  3-clause        [[  4-clause_atom   (     1.0 choices)
edge  4-clause_atom   [[  5-phrase        (     1.0 choic

The part about the *nodes* shows you how many possible instantiations for each object in your template
has been found.
These are not results yet, because only combinations of instantiations
that satisfy all constraints are results.

The constraints come from the relations between the objects that you specified.
In this case, there is only an implicit relation: embedding `[[`. 
Later on we'll examine all basic relations.

The part about the *edges* shows you the constraints,
and in what order they will be computed when stitching results together.
In this case the order is exactly the order by which the relations appear in the template,
but that will not always be the case.
Text-Fabric spends some time and ingenuity to find out an optimal *stitch plan*.

Nevertheless, fetching results may take time. 

For some queries, it can take a large amount of time to walk through all results.
Even worse, it may happen that it takes a large amount of time before getting the *first* result.

This has to do with search strategies on the one hand,
and the very likely possibility to encounter *pathological* search patterns,
which have billions of results, mostly unintended.
For example, a simple query that asks for 5 words in the Hebrew Bible without further constraints,
will have 425,000 to the power of 5 results.
That is 10-e28 (a one with 28 zeroes),
roughly the number of molecules in a few hundred litres of air.
That may not sound much, but it is 10,000 times the amount of bytes
that can be currently stored on the whole internet.

Text-Fabric search is not yet done with finding optimal search strategies,
and I hope to refine its arsenal of methods in the future, depending on what you report.

## Back to business
It is always a good idea to get a feel for the amount of results, before you dive into them head-on.

In [9]:
S.count(progress=1, limit=5)

  0.00s Counting results per 1 up to 5 ...
   |     0.00s 1
   |     0.00s 2
   |     0.00s 3
   |     0.01s 4
   |     0.01s 5
  0.01s Done: 5 results


We asked for 5 results in total, with a progress message for every one.
That was a bit conservative.

In [10]:
S.count(progress=100, limit=500)

  0.00s Counting results per 100 up to 500 ...
   |     0.02s 100
   |     0.04s 200
   |     0.05s 300
   |     0.09s 400
   |     0.13s 500
  0.14s Done: 500 results


Still pretty quick, now we want to count all results.

In [11]:
S.count(progress=200, limit=-1)

  0.00s Counting results per 200 up to  the end of the results ...
   |     0.03s 200
   |     0.09s 400
   |     0.13s 600
   |     0.17s 800
  0.20s Done: 926 results


It is time to see something of those results.

In [12]:
S.fetch(amount=10)

((1367552, 1368104, 1428265, 486532, 576266, 781925, 1045061, 299521),
 (1367553, 1368106, 1428282, 486600, 576336, 782114, 1045258, 299826),
 (1367553, 1368107, 1428301, 486694, 576432, 782371, 1045520, 300192),
 (1367553, 1368108, 1428315, 486756, 576495, 782555, 1045709, 300482),
 (1367553, 1368108, 1428310, 486735, 576473, 782501, 1045653, 300389),
 (1367553, 1368109, 1428327, 486823, 576563, 782738, 1045900, 300764),
 (1367553, 1368111, 1428352, 486915, 576657, 782998, 1046167, 301139),
 (1367553, 1368111, 1428351, 486910, 576652, 782986, 1046155, 301122),
 (1367554, 1368113, 1428393, 487091, 576838, 783442, 1046621, 301774),
 (1367554, 1368113, 1428394, 487094, 576841, 783449, 1046628, 301781))

Not very informative.
Just a quick observation: look at the last column.
These are the result nodes for the `word` part in the query, indicated as `R7` by `showPlan()` before.
And indeed, they are all below 425,000, the number of words in the Hebrew Bible.

Nevertheless, we want to glean a bit more information off them.

In [13]:
for r in S.fetch(amount=10):
    print(S.glean(r))

  Jonah 4:11 clause[אֲשֶׁ֣ר יֶשׁ־בָּ֡הּ ] clause_atom[אֲשֶׁ֣ר יֶשׁ־בָּ֡הּ ] phrase[יֶשׁ־] phrase_atom[יֶשׁ־] יֶשׁ־
  Micah 2:1 clause[כִּ֥י יֶשׁ־לְאֵ֖ל יָדָֽם׃ ] clause_atom[כִּ֥י יֶשׁ־לְאֵ֖ל יָדָֽם׃ ] phrase[יֶשׁ־] phrase_atom[יֶשׁ־] יֶשׁ־
  Micah 3:7 clause[כִּ֛י אֵ֥ין מַעֲנֵ֖ה אֱלֹהִֽים׃ ] clause_atom[כִּ֛י אֵ֥ין מַעֲנֵ֖ה אֱלֹהִֽים׃ ] phrase[אֵ֥ין ] phrase_atom[אֵ֥ין ] אֵ֥ין 
  Micah 4:9 clause[הֲמֶ֣לֶךְ אֵֽין־בָּ֗ךְ ] clause_atom[הֲמֶ֣לֶךְ אֵֽין־בָּ֗ךְ ] phrase[אֵֽין־] phrase_atom[אֵֽין־] אֵֽין־
  Micah 4:4 clause[וְאֵ֣ין מַחֲרִ֑יד ] clause_atom[וְאֵ֣ין מַחֲרִ֑יד ] phrase[אֵ֣ין ] phrase_atom[אֵ֣ין ] אֵ֣ין 
  Micah 5:7 clause[וְאֵ֥ין מַצִּֽיל׃ ] clause_atom[וְאֵ֥ין מַצִּֽיל׃ ] phrase[אֵ֥ין ] phrase_atom[אֵ֥ין ] אֵ֥ין 
  Micah 7:2 clause[וְיָשָׁ֥ר בָּאָדָ֖ם ...] clause_atom[וְיָשָׁ֥ר בָּאָדָ֖ם ...] phrase[אָ֑יִן ] phrase_atom[אָ֑יִן ] אָ֑יִן 
  Micah 7:1 clause[אֵין־אֶשְׁכֹּ֣ול ] clause_atom[אֵין־אֶשְׁכֹּ֣ול ] phrase[אֵין־] phrase_atom[אֵין־] אֵין־
  Nahum 2:9 clause[וְאֵ֥ין מַפְנֶֽה

##### Caution
> It is not possible to do `len(S.fetch())`.
Because `fetch()` is a *generator*, not a list.
It will deliver a result every time it is being asked and for as long as there are results,
but it does not know in advance how many there will be.

>Fetching a result can be costly, because due to the constraints, a lot of possibilities
may have to be tried and rejected before a the next result is found.

> That is why you often see results coming in at varying speeds when counting them.

This search template has some pretty tight constraints on one of its objects,
so the amount of data to deal with is pretty limited.

Let us turn to a template where this is not so.

In [14]:
query = '''
# test
# verse book=Genesis chapter=2 verse=25
verse
  clause
                                 
    p1:phrase
        w1:word
        w3:word
        w1 < w3

    p2:phrase
        w2:word
        w1 < w2 
        w3 > w2
    
    p1 < p2   
'''

A couple of remarks.

* some objects have got a name
* there are additional relations specified between named objects
* `<` means: *comes before*, and `>`: *comes after* in the canonical order for nodes,
  which for words means: comes textually before/after, but for other nodes the meaning
  is explained [here](https://github.com/ETCBC/text-fabric/wiki/Api#sorting-nodes)
* later on we describe those relations in more detail

##### Note on order
> Look at the words `w1` and `w3` below phrase `p1`.
Although in the template `w1` comes before `w3`, this is not 
translated in a search constraint of the same nature.

> Order between objects in a template is never significant, only embedding is.

Because order is not significant, you have to specify order yourself, using relations.

It turns out that this is better than the other way around.
In MQL order *is* significant, and it is very difficult to 
search for `w1` and `w2` in any order.
Especially if your are looking for more than 2 complex objects with lots of feature
conditions, your search template would explode if you had to spell out all
possible permutations. See the example of Reinoud Oosting below.

##### Note on gaps
> Look at the phrases `p1` and `p2`.
We do not specify an order here, only that they are different.
In order to prevent duplicated searches with `p1` and `p2` interchanged, we even 
stipulate that `p1 < p2`.
There are many spatial relationships possible between different objects.
In many cases, neither the one comes before the other, nor vice versa.
They can overlap, one can occur in a gap of the other, they can be completely disjoint
and interleaved, etc.

In [15]:
S.study(query)

  0.00s Checking search template ...
  0.00s Setting up search space for 7 objects ...
  0.50s Constraining search space with 10 relations ...
  0.55s Setting up retrieval plan ...
  0.60s Ready to deliver results from 1897304 nodes
Iterate over S.fetch() to get the results
See S.showPlan() to interpret the results


That was quick!
Well, Text-Fabric knows that narrowing down the search space in this case would take ages,
without resulting in a significantly shrunken space.
So it skips doing so for most constraints.

Let us see the plan, with details.

In [16]:
S.showPlan(details=True)

Search with 7 objects and 10 relations
Results are instantiations of the following objects:
node  0-verse                             ( 23213   choices)
node  1-clause                            ( 88000   choices)
node  2-phrase                            (253174   choices)
node  3-word                              (426581   choices)
node  4-word                              (426581   choices)
node  5-phrase                            (253174   choices)
node  6-word                              (426581   choices)
Instantiations are computed along the following relations:
node                      0-verse         ( 23213   choices)
edge  0-verse         [[  1-clause        (     3.7 choices)
edge  1-clause        [[  5-phrase        (     3.1 choices)
edge  5-phrase        [[  6-word          (     1.6 choices)
edge  1-clause        [[  2-phrase        (     3.4 choices)
edge  2-phrase        <   5-phrase        (126587.0 choices)
edge  2-phrase        [[  4-word          (     1.5 choi

As you see, we have a hefty search space here.
Let us play with the `count()` function.

In [17]:
S.count(progress=10, limit=100)

  0.00s Counting results per 10 up to 100 ...
   |     0.09s 10
   |     0.10s 20
   |     0.10s 30
   |     0.12s 40
   |     0.12s 50
   |     0.13s 60
   |     0.14s 70
   |     0.14s 80
   |     0.15s 90
   |     0.15s 100
  0.15s Done: 100 results


We can be bolder than this!

In [18]:
S.count(progress=100, limit=1000)

  0.00s Counting results per 100 up to 1000 ...
   |     0.12s 100
   |     0.15s 200
   |     0.15s 300
   |     0.29s 400
   |     0.36s 500
   |     0.37s 600
   |     0.47s 700
   |     0.55s 800
   |     0.66s 900
   |     0.72s 1000
  0.72s Done: 1000 results


Ok, not too bad, but note that it takes a big fraction of a second to get just 100 results.

Now let us go for all of them by the thousand.

In [19]:
S.count(progress=1000, limit=-1)

  0.00s Counting results per 1000 up to  the end of the results ...
   |     0.66s 1000
   |     1.02s 2000
   |     1.55s 3000
   |     1.78s 4000
   |     2.43s 5000
   |     3.49s 6000
   |     5.32s 7000
  6.01s Done: 7512 results


See? This is substantial work.

In [20]:
for r in S.fetch(amount=10):
    print(S.glean(r))

Genesis 2:25 clause[וַיִּֽהְי֤וּ שְׁנֵיהֶם֙ עֲרוּמִּ֔ים הָֽ...] phrase[שְׁנֵיהֶם֙ הָֽאָדָ֖ם וְאִשְׁתֹּ֑ו ] שְׁנֵיהֶם֙  הָֽ phrase[עֲרוּמִּ֔ים ] עֲרוּמִּ֔ים 
Genesis 2:25 clause[וַיִּֽהְי֤וּ שְׁנֵיהֶם֙ עֲרוּמִּ֔ים הָֽ...] phrase[שְׁנֵיהֶם֙ הָֽאָדָ֖ם וְאִשְׁתֹּ֑ו ] שְׁנֵיהֶם֙  אָדָ֖ם  phrase[עֲרוּמִּ֔ים ] עֲרוּמִּ֔ים 
Genesis 2:25 clause[וַיִּֽהְי֤וּ שְׁנֵיהֶם֙ עֲרוּמִּ֔ים הָֽ...] phrase[שְׁנֵיהֶם֙ הָֽאָדָ֖ם וְאִשְׁתֹּ֑ו ] שְׁנֵיהֶם֙  וְ phrase[עֲרוּמִּ֔ים ] עֲרוּמִּ֔ים 
Genesis 2:25 clause[וַיִּֽהְי֤וּ שְׁנֵיהֶם֙ עֲרוּמִּ֔ים הָֽ...] phrase[שְׁנֵיהֶם֙ הָֽאָדָ֖ם וְאִשְׁתֹּ֑ו ] שְׁנֵיהֶם֙  אִשְׁתֹּ֑ו  phrase[עֲרוּמִּ֔ים ] עֲרוּמִּ֔ים 
Genesis 4:4 clause[וְהֶ֨בֶל הֵבִ֥יא גַם־ה֛וּא ...] phrase[הֶ֨בֶל גַם־ה֛וּא ] הֶ֨בֶל  גַם־ phrase[הֵבִ֥יא ] הֵבִ֥יא 
Genesis 4:4 clause[וְהֶ֨בֶל הֵבִ֥יא גַם־ה֛וּא ...] phrase[הֶ֨בֶל גַם־ה֛וּא ] הֶ֨בֶל  ה֛וּא  phrase[הֵבִ֥יא ] הֵבִ֥יא 
Genesis 10:21 clause[גַּם־ה֑וּא אֲבִי֙ כָּל־בְּנֵי־...] phrase[גַּם־ה֑וּא אֲחִ֖י יֶ֥פֶת הַ...] גַּם־ אֲחִ֖י  phrase[אֲבִי֙ כָּל

As a check, here is some code that looks for basically the same phenomenon:
a phrase within the gap of another phrase.
It does not use search, and it gets a bit more focused results, in half the time compared
to the search with the template.

##### Hint
> If you are comfortable with programming, and what you look for is fairly generic,
you may be better off without search, provided you can translate your insight in the
data into an effective procedure within Text-Fabric.
But wait till we are completely done with this example!

In [21]:
indent(reset=True)
info('Getting gapped phrases')
results = []
for v in F.otype.s('verse'):
    for c in L.d(v, otype='clause'):
        ps = L.d(c, otype='phrase')
        first = {}
        last = {}
        slots = {}
        # make index of phrase boundaries
        for p in ps:
            words = L.d(p, otype='word')
            first[p] = words[0]
            last[p] = words[-1]
            slots[p] = set(words)
        for p1 in ps:
            for p2 in ps:
                if p2 < p1: continue
                if len(slots[p1] & slots[p2]) != 0: continue
                if first[p1] < first[p2] and last[p2] < last[p1]:
                    results.append((v, c, p1, p2, first[p1], first[p2], last[p2], last[p1]))
info('{} results'.format(len(results)))
for r in results[0:10]:
    print(r)

  0.00s Getting gapped phrases
  3.40s 369 results
(1413737, 426799, 605793, 605794, 1159, 1160, 1160, 1164)
(1413765, 426921, 606150, 606151, 1720, 1721, 1721, 1723)
(1413937, 427418, 607746, 607747, 4819, 4821, 4824, 4828)
(1413997, 427601, 608322, 608323, 5803, 5805, 5806, 5809)
(1414001, 427616, 608369, 608370, 5868, 5869, 5870, 5875)
(1414034, 427723, 608705, 608706, 6515, 6521, 6521, 6530)
(1414086, 427917, 609286, 609287, 7431, 7432, 7433, 7437)
(1414143, 428159, 609997, 609998, 8502, 8507, 8507, 8520)
(1414143, 428159, 609997, 609999, 8502, 8508, 8510, 8520)
(1414172, 428286, 610379, 610380, 9127, 9129, 9129, 9133)


But we can use the pretty printing of `glean()` here as well, even though we have
not used search!

In [22]:
for r in results[0:10]:
    print(S.glean(r))

Genesis 2:25 clause[וַיִּֽהְי֤וּ שְׁנֵיהֶם֙ עֲרוּמִּ֔ים הָֽ...] phrase[שְׁנֵיהֶם֙ הָֽאָדָ֖ם וְאִשְׁתֹּ֑ו ] phrase[עֲרוּמִּ֔ים ] שְׁנֵיהֶם֙  עֲרוּמִּ֔ים  עֲרוּמִּ֔ים  אִשְׁתֹּ֑ו 
Genesis 4:4 clause[וְהֶ֨בֶל הֵבִ֥יא גַם־ה֛וּא ...] phrase[הֶ֨בֶל גַם־ה֛וּא ] phrase[הֵבִ֥יא ] הֶ֨בֶל  הֵבִ֥יא  הֵבִ֥יא  ה֛וּא 
Genesis 10:21 clause[גַּם־ה֑וּא אֲבִי֙ כָּל־בְּנֵי־...] phrase[גַּם־ה֑וּא אֲחִ֖י יֶ֥פֶת הַ...] phrase[אֲבִי֙ כָּל־בְּנֵי־עֵ֔בֶר ] גַּם־ אֲבִי֙  עֵ֔בֶר  גָּדֹֽול׃ 
Genesis 12:17 clause[וַיְנַגַּ֨ע יְהוָ֧ה׀ אֶת־פַּרְעֹ֛ה ...] phrase[אֶת־פַּרְעֹ֛ה וְאֶת־בֵּיתֹ֑ו ] phrase[נְגָעִ֥ים גְּדֹלִ֖ים ] אֶת־ נְגָעִ֥ים  גְּדֹלִ֖ים  בֵּיתֹ֑ו 
Genesis 13:1 clause[וַיַּעַל֩ אַבְרָ֨ם מִמִּצְרַ֜יִם ...] phrase[אַבְרָ֨ם ה֠וּא וְאִשְׁתֹּ֧ו וְ...] phrase[מִמִּצְרַ֜יִם ] אַבְרָ֨ם  מִ מִּצְרַ֜יִם  כָל־
Genesis 14:16 clause[וְגַם֩ אֶת־לֹ֨וט אָחִ֤יו ...] phrase[גַם֩ אֶת־לֹ֨וט אָחִ֤יו וּ...] phrase[הֵשִׁ֔יב ] גַם֩  הֵשִׁ֔יב  הֵשִׁ֔יב  עָֽם׃ 
Genesis 17:7 clause[לִהְיֹ֤ות לְךָ֙ לֵֽאלֹהִ֔ים ...] phrase[לְךָ֙ וּֽלְז

# Refine the search template

A second look at the results of our search template reveals 
that there are multiple results per pair of phrases,
because there are in general multiple words in both
phrases that satisfy the condition.
We can make the search template stricter,
by requiring alignment of the words with the starts and ends of the phrases
they are in.

For this, we employ a convenient device in search templates that we have not explained yet.

**Before each atom we may put a relational operator.**

The meaning is that this relation holds between the preceding atom and the current one.
If there is a lonely operator all by itself on a line, it means that 
this relation holds between the preceding sibling and the parent.

These operators are very handy to indicate that there is an order between siblings,
and also that a child should start or end where the parent starts or ends.

In [23]:
query = '''
verse
  clause
                                 
    p1:phrase
        =: w1:word
        <  w3:word
        :=

    p2:phrase
        =: w2:word
    
    p1 < p2
    w1 < p2
    w2 < w3
    
'''

The line 

```
=: w1:word
``` 

constrains word `w1` to start exactly at the start of its parent, phrase `p1`.

The line

```
<  w3:word
```

constrains the preceding sibling `w1` to come before `w3` in the canonical node ordering.
Because `w1` and `w3` are words, this means that `w1` comes textually before `w3`.

The line 

```
:=
``` 

constrains the preceding sibling, word `w3` to end exactly at the end of its parent,
phrase `p1`.

The line 

```
=: w2:word
```

constrains word `w2` to start exactly at the start of its parent, phrase `p2`.

Given two phrases `p1` and `p2`, the positions of all three words `w1`, `w2`, `w3` are fixed, so for every pair `p1`, `p2` that satisfies the conditions, 
there is exactly one result.

In [24]:
S.study(query)
S.showPlan(details=True)
S.count(progress=100, limit=-1)
for r in S.fetch(amount=10):
    print(S.glean(r))

  0.00s Checking search template ...
  0.00s Setting up search space for 7 objects ...
  0.50s Constraining search space with 13 relations ...
  0.56s Setting up retrieval plan ...
  0.68s Ready to deliver results from 1897304 nodes
Iterate over S.fetch() to get the results
See S.showPlan() to interpret the results
Search with 7 objects and 13 relations
Results are instantiations of the following objects:
node  0-verse                             ( 23213   choices)
node  1-clause                            ( 88000   choices)
node  2-phrase                            (253174   choices)
node  3-word                              (426581   choices)
node  4-word                              (426581   choices)
node  5-phrase                            (253174   choices)
node  6-word                              (426581   choices)
Instantiations are computed along the following relations:
node                      0-verse         ( 23213   choices)
edge  0-verse         [[  1-clause        ( 

And here we have exactly the same results as our hand-written piece of code.

> Note
Now, with the "duplicate" results prevented, the search with the template has
only a slight performance overhead compared to the manual piece of code!

But beware of complications. 
Search templates are powerful, but sometimes they lead to a different
result set from what you might think.
Here is an example.

# A tricky example

Suppose we want to count the clauses consisting of exactly two phrases.
The following template should do it:
a clause, starting with a phrase, followed by an adjacent phrase,
which terminates the clause.

In [25]:
query = '''
clause
    =: phrase
    <: phrase
    :=
'''

In [26]:
S.study(query)
S.showPlan()
qresults = sorted(r[0] for r in S.fetch())
info(f'Done: found {len(qresults)}')

  0.00s Checking search template ...
  0.00s Setting up search space for 3 objects ...
  0.17s Constraining search space with 5 relations ...
  0.20s Setting up retrieval plan ...
  0.25s Ready to deliver results from 594348 nodes
Iterate over S.fetch() to get the results
See S.showPlan() to interpret the results
  0.25s The results are connected to the original search template as follows:
 0     
 1 R0  clause
 2 R1      =: phrase
 3 R2      <: phrase
 4         :=
 5     
  1.18s Done: found 23415


Let us check this with a piece of hand-written code.

In [27]:
indent(reset=True)
info('counting ...')

cresults = []
for c in F.otype.s('clause'):
    wc = L.d(c, otype='word')
    ps = L.d(c, otype='phrase')
    if len(ps) == 2:
        (fp, lp) = ps
        wf = L.d(fp, otype='word')
        wl = L.d(lp, otype='word')
        if wf[0] == wc[0] and wf[-1] + 1 == wl[0] and wl[-1] == wc[-1]:
            cresults.append(c)
cresults = sorted(cresults)
info(f'Done: found {len(cresults)}')

  0.00s counting ...
  1.46s Done: found 23332


Strange, we end up with less cases. What is happening? Let us compare the results.
We look at the first result where both methods diverge.

In [28]:
diff = [x for x in zip(qresults, cresults) if x[0] != x[1]]
print(f'{len(diff)} differences')
print(diff[0])

23053 differences
(427723, 427728)


Let's look at the phrases of 427723:

In [29]:
for p in L.d(diff[0][0], otype='phrase'):
    print(f'Phrase {p} has words {L.d(p, otype="word")}')

Phrase 608704 has words [6514]
Phrase 608705 has words [6515, 6516, 6517, 6518, 6519, 6520, 6522, 6523, 6524, 6525, 6526, 6527, 6528, 6529, 6530]
Phrase 608706 has words [6521]


This clause has three phrases, but the third one lies inside the second one,
so that the clause indeed satisfies the pattern of two adjacent phrases.

Can we adjust the pattern to exclude cases like this? 
At the moment, our search template mechanism is not powerful enough for that.

We can count how often it happens, however. 
We require a third phrase to be present, not equal to one of the first two ones.

In [30]:
query = '''
clause
    =: p1:phrase
    <: p2:phrase
    :=
    p3:phrase
    p1 # p3
    p2 # p3
'''

In [31]:
S.study(query)
S.showPlan()
rresults = sorted(r[0] for r in S.fetch())
info(f'Done: found {len(rresults)}')

  0.00s Checking search template ...
  0.00s Setting up search space for 4 objects ...
  0.25s Constraining search space with 8 relations ...
  0.27s Setting up retrieval plan ...
  0.34s Ready to deliver results from 847522 nodes
Iterate over S.fetch() to get the results
See S.showPlan() to interpret the results
  0.34s The results are connected to the original search template as follows:
 0     
 1 R0  clause
 2 R1      =: p1:phrase
 3 R2      <: p2:phrase
 4         :=
 5 R3      p3:phrase
 6         p1 # p3
 7         p2 # p3
 8     
  1.38s Done: found 115


But we have to filter this, because per `p1`, `p2` there might be multiple `p3` that satisfy the constraints.
So lets gather the set of `p1`, `p2` pairs.

In [32]:
len(set(rresults))

83

And this is exactly the difference between 
the number of results of the search template and the hand-written piece of code.

# Testing basic relations

Basic relations are about the identity spatial ordering of objects.
Are they the same, do they occupy the same slots, do they overlap, is one embedded in the other,
does one come before the other?

We also have edge features, that specify relationships between nodes.

Although the basic relationships are easy to define, and even easy to implement,
they may be very costly to use. 
When searching, most of them have to be computed very many times.

Some of them have been precomputed and stored in an index, e.g. the embedding relationships.
They can be used without penalty.

Other relations are not suitable for precomputing: most inequality relations are of that kind.
It would require an enormous amount of storage to precompute for each node the set of nodes that
occupy different slots. This type of relation will not be used in narrowing down the search space,
which means that it may take more time to get the results.

We are going to test all of our basic relationships here.

Let us first see what relations we have:

In [33]:
print(S.relationLegend)

                      = left equal to right (as node)
                      # left unequal to right (as node)
                      < left before right (in canonical node ordering)
                      > left after right (in canonical node ordering)
                     == left occupies same slots as right
                     && left has overlapping slots with right
                     ## left and right do not have the same slot set
                     || left and right do not have common slots
                     [[ left embeds right
                     ]] left embedded in right
                     << left completely before right
                     >> left completely after right
                     =: left and right start at the same slot
                     := left and right end at the same slot
                     :: left and right start and end at the same slot
                     <: left immediately before right
                     :> left immediately after right
   

# = (equal as node)

The `=` means that both parts are the same node. Left and right are not two things with similar properties,
no, they are one and the same thing.

Useful if the thing you search for it part of two wildly different patterns.

In [34]:
query = '''
v1:verse
  sentence
    clause rela=Objc
      phrase
        word sp=verb gn=f nu=pl
v2:verse
  sentence
    c1:clause
    c2:clause
    c3:clause
    c1 < c2
    c2 < c3
v1 = v2
'''
S.study(query)
S.showPlan()
for r in S.fetch(amount=10):
    print(S.glean(r))

  0.00s Checking search template ...
  0.00s loading features ...
   |     0.17s B gn                   from /Users/dirk/github/text-fabric-data/hebrew/etcbc4c
   |     0.14s B nu                   from /Users/dirk/github/text-fabric-data/hebrew/etcbc4c
   |     0.23s B rela                 from /Users/dirk/github/text-fabric-data/hebrew/etcbc4c
   |     0.14s B sp                   from /Users/dirk/github/text-fabric-data/hebrew/etcbc4c
  0.69s All additional features loaded - for details use loadLog()
  0.69s Setting up search space for 10 objects ...
  1.64s Constraining search space with 11 relations ...
  2.20s Setting up retrieval plan ...
  2.23s Ready to deliver results from 327603 nodes
Iterate over S.fetch() to get the results
See S.showPlan() to interpret the results
  2.24s The results are connected to the original search template as follows:
 0     
 1 R0  v1:verse
 2 R1    sentence
 3 R2      clause rela=Objc
 4 R3        phrase
 5 R4          word sp=verb gn=f nu=pl
 6 R

# # (unequal as node)

`n # m` if `n` and `m` are not the same node.

If you write a template, and you know that one node should come before another one,
consider using `<` or `>`, which will constrain the results better.

We have seen this in action in the search for gapped phrases.

# < and > (canonical)

`n < m` if `n` comes before `m` in the
[canonical ordering](https://github.com/ETCBC/text-fabric/wiki/Api#sorting-nodes)
of nodes.

We have seen them in action before.

# == (same slots)

Two objects are extensionally equal if they occupy exactly the same slots.

Quite an expensive relation, as you will see: nearly 30 seconds for 3608 results.

In [35]:
query = '''
v:verse
    s:sentence
v == s
'''
S.study(query)
S.showPlan()
for r in S.fetch(amount=10):
    print(S.glean(r))
S.count(progress=1000, limit=10000)

  0.00s Checking search template ...
  0.00s Setting up search space for 2 objects ...
  0.03s Constraining search space with 2 relations ...
  0.93s Setting up retrieval plan ...
  0.96s Ready to deliver results from 7216 nodes
Iterate over S.fetch() to get the results
See S.showPlan() to interpret the results
  0.96s The results are connected to the original search template as follows:
 0     
 1 R0  v:verse
 2 R1      s:sentence
 3     v == s
 4     
Jeremiah 10:17 sentence[אִסְפִּ֥י מֵאֶ֖רֶץ כִּנְעָתֵ֑ךְ יֹשֶׁ֖בֶת ...]
Jeremiah 10:22 sentence[קֹ֤ול שְׁמוּעָה֙ הִנֵּ֣ה בָאָ֔ה וְ...]
Jeremiah 10:23 sentence[יָדַ֣עְתִּי יְהוָ֔ה כִּ֛י לֹ֥א לָ...]
Jeremiah 11:1 sentence[הַדָּבָר֙ אֲשֶׁ֣ר הָיָ֣ה אֶֽל־...]
Jeremiah 11:17 sentence[וַיהוָ֤ה צְבָאֹות֙ הַנֹּוטֵ֣עַ ...]
Jeremiah 13:3 sentence[וַיְהִ֧י דְבַר־יְהוָ֛ה אֵלַ֖י ...]
Jeremiah 13:8 sentence[וַיְהִ֥י דְבַר־יְהוָ֖ה אֵלַ֥י ...]
Jeremiah 13:10 sentence[הָעָם֩ הַזֶּ֨ה הָ...]
Jeremiah 13:24 sentence[וַאֲפִיצֵ֖ם כְּקַשׁ־עֹובֵ֑ר ...]
Jeremiah 

# && (overlap)

Two objects overlap if and only if they share at least one slot.
This is quite costly to use in some cases.

In [37]:
query = '''
verse
    phrase
      s1:subphrase
      s2:subphrase
      s1 # s2
      s1 && s2
'''
S.study(query)
S.showPlan()
for r in S.fetch(amount=10):
    print(S.glean(r))

  0.00s Checking search template ...
  0.00s Setting up search space for 4 objects ...
  0.17s Constraining search space with 5 relations ...
  0.85s Setting up retrieval plan ...
  1.17s Ready to deliver results from 503971 nodes
Iterate over S.fetch() to get the results
See S.showPlan() to interpret the results
  1.18s The results are connected to the original search template as follows:
 0     
 1 R0  verse
 2 R1      phrase
 3 R2        s1:subphrase
 4 R3        s2:subphrase
 5           s1 # s2
 6           s1 && s2
 7     
Genesis 1:14 phrase[לְאֹתֹת֙ וּלְמֹ֣ועֲדִ֔ים ...] subphrase[לְאֹתֹת֙ ] subphrase[לְאֹתֹת֙ וּלְמֹ֣ועֲדִ֔ים ]
Genesis 1:14 phrase[לְאֹתֹת֙ וּלְמֹ֣ועֲדִ֔ים ...] subphrase[לְאֹתֹת֙ וּלְמֹ֣ועֲדִ֔ים ] subphrase[לְאֹתֹת֙ ]
Genesis 1:14 phrase[לְאֹתֹת֙ וּלְמֹ֣ועֲדִ֔ים ...] subphrase[לְאֹתֹת֙ וּלְמֹ֣ועֲדִ֔ים ] subphrase[לְמֹ֣ועֲדִ֔ים ]
Genesis 1:14 phrase[לְאֹתֹת֙ וּלְמֹ֣ועֲדִ֔ים ...] subphrase[לְמֹ֣ועֲדִ֔ים ] subphrase[לְאֹתֹת֙ וּלְמֹ֣ועֲדִ֔ים ]
Genesis 1:14 phrase[לְא

# ## (not the same slots)

True when the two objects in question do not occupy exactly the same set of slots.
This is a very loose relationship.

It is implemented, but not yet tested, and at the moment I have not a clear use case for it.

# || (disjoint slots)

True when the two objects in question do not share any slots.
This is a rather loose relationship.

It is implemented, but not yet tested, and at the moment I have not a clear use case for it.

# [[ and ]] (embedding)

`n [[ m` if object `n` embeds `m`.

`n ]] m` if object `n` lies embedded in `n`.

These relations are used implicitly in templates when there is indentation:

```
s:sentence
  p:phrase
    w1:word gn=f
    w2:word gn=m
```

The template above implicitly states the following embeddings:

* `s ]] p`
* `p ]] w1`
* `p ]] w2`

We have seen these relations in action.

# << and >> (before and after with slots)

These relations test whether one object comes before or after an other,
in the sense that the slots
occupied by the one object lie completely 
before or after the slots occupied by the other object.

In [38]:
query = '''
verse
  sentence
    c1:clause
    p:phrase
    c2:clause
    c1 << p
    c2 >> p
'''
S.study(query)
S.showPlan()
for r in S.fetch(amount=10):
    print(S.glean(r))

  0.00s Checking search template ...
  0.00s Setting up search space for 5 objects ...
  0.17s Constraining search space with 6 relations ...
  0.18s Setting up retrieval plan ...
  0.20s Ready to deliver results from 515957 nodes
Iterate over S.fetch() to get the results
See S.showPlan() to interpret the results
  0.21s The results are connected to the original search template as follows:
 0     
 1 R0  verse
 2 R1    sentence
 3 R2      c1:clause
 4 R3      p:phrase
 5 R4      c2:clause
 6         c1 << p
 7         c2 >> p
 8     
Genesis 1:11 sentence[תַּֽדְשֵׁ֤א הָאָ֨רֶץ֙ דֶּ֔שֶׁא עֵ֚שֶׂב ...] clause[תַּֽדְשֵׁ֤א הָאָ֨רֶץ֙ דֶּ֔שֶׁא עֵ֚שֶׂב ...] phrase[עֹ֤שֶׂה ] clause[אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו ]
Genesis 1:11 sentence[תַּֽדְשֵׁ֤א הָאָ֨רֶץ֙ דֶּ֔שֶׁא עֵ֚שֶׂב ...] clause[תַּֽדְשֵׁ֤א הָאָ֨רֶץ֙ דֶּ֔שֶׁא עֵ֚שֶׂב ...] phrase[פְּרִי֙ ] clause[אֲשֶׁ֥ר זַרְעֹו־בֹ֖ו ]
Genesis 1:11 sentence[תַּֽדְשֵׁ֤א הָאָ֨רֶץ֙ דֶּ֔שֶׁא עֵ֚שֶׂב ...] clause[תַּֽדְשֵׁ֤א הָאָ֨רֶץ֙ דֶּ֔שֶׁא עֵ֚שֶׂב ...] phrase[לְמִינֹ֔

# =: (start at same slots)
This relation holds when the left and right hand sides are nodes that have the same first slot.
It serves to enforce the the children of a parent are textually the first things inside that
parent. We have seen it in action before.

# := (end at same slots)
This relation holds when the left and right hand sides are nodes that have the same last slot
It serves to enforce the the children of a parent are textually the last things inside that
parent. We have seen it in action before.

# :: (same start and end slots)
This relation holds when `=:` and `:=` both hold between the left and right hand sides.
It serves to look for parents with single children, or at least, where the parent is textually spanned by a single child.

In [39]:
query = '''
verse
    clause
        :: phrase
'''
S.study(query)
S.showPlan()
S.count(progress=1000, limit=-1)
for r in S.fetch(amount=10):
    print(S.glean(r))

  0.00s Checking search template ...
  0.00s Setting up search space for 3 objects ...
  0.13s Constraining search space with 3 relations ...
  0.14s Setting up retrieval plan ...
  0.18s Ready to deliver results from 364387 nodes
Iterate over S.fetch() to get the results
See S.showPlan() to interpret the results
  0.18s The results are connected to the original search template as follows:
 0     
 1 R0  verse
 2 R1      clause
 3 R2          :: phrase
 4     
  0.00s Counting results per 1000 up to  the end of the results ...
   |     0.14s 1000
   |     0.26s 2000
   |     0.34s 3000
   |     0.40s 4000
   |     0.46s 5000
   |     0.52s 6000
   |     0.56s 7000
   |     0.62s 8000
   |     0.69s 9000
  0.73s Done: 9387 results
Genesis 1:5 clause[יֹ֥ום אֶחָֽד׃ פ ] phrase[יֹ֥ום אֶחָֽד׃ פ ]
Genesis 1:8 clause[יֹ֥ום שֵׁנִֽי׃ פ ] phrase[יֹ֥ום שֵׁנִֽי׃ פ ]
Genesis 1:13 clause[יֹ֥ום שְׁלִישִֽׁי׃ פ ] phrase[יֹ֥ום שְׁלִישִֽׁי׃ פ ]
Genesis 1:16 clause[אֶת־הַמָּאֹ֤ור הַגָּדֹל֙ ...] phrase[אֶת־

Like before, there might be extra phrases in such clauses, lying embedded in the clause-spanning phrase.

In [40]:
query = '''
verse
    clause
        :: p1:phrase
        p2:phrase
        p1 # p2
'''
S.study(query)
S.showPlan()
S.count(progress=1000, limit=-1)
for r in S.fetch(amount=10):
    print(S.glean(r))

  0.00s Checking search template ...
  0.00s Setting up search space for 4 objects ...
  0.21s Constraining search space with 5 relations ...
  0.54s Setting up retrieval plan ...
  0.55s Ready to deliver results from 295167 nodes
Iterate over S.fetch() to get the results
See S.showPlan() to interpret the results
  0.55s The results are connected to the original search template as follows:
 0     
 1 R0  verse
 2 R1      clause
 3 R2          :: p1:phrase
 4 R3          p2:phrase
 5             p1 # p2
 6     
  0.00s Counting results per 1000 up to  the end of the results ...
  0.18s Done: 82 results
2_Samuel 22:31 clause[מָגֵ֣ן ה֔וּא לְכֹ֖ל ] phrase[מָגֵ֣ן לְכֹ֖ל ] phrase[ה֔וּא ]
Genesis 10:21 clause[גַּם־ה֑וּא אֲבִי֙ כָּל־בְּנֵי־...] phrase[גַּם־ה֑וּא אֲחִ֖י יֶ֥פֶת הַ...] phrase[אֲבִי֙ כָּל־בְּנֵי־עֵ֔בֶר ]
1_Kings 2:26 clause[עֲנָתֹת֙ לֵ֣ךְ עַל־שָׂדֶ֔יךָ ] phrase[עֲנָתֹת֙ עַל־שָׂדֶ֔יךָ ] phrase[לֵ֣ךְ ]
Genesis 24:24 clause[בַּת־בְּתוּאֵ֖ל אָנֹ֑כִי בֶּן־מִלְכָּ֕ה ] phrase[בַּת־בְּתוּ

# <: (adjacent before) 
This relation holds when the left hand sides ends in a slot that lies before the first slot of the right hand side.
It serves to enforce an ordering between siblings of a parent.

# :> (adjacent after)
This relation holds when the left hand sides starts in a slot that lies after the last slot of the right hand side.

As an example: are there clauses with multiple clause atoms without a gap between the two?

In [41]:
query = '''
verse
    clause
        clause_atom
        <: clause_atom
'''
S.study(query)
S.showPlan()
S.count(progress=1000, limit=-1)
for r in S.fetch(amount=10):
    print(S.glean(r))

  0.00s Checking search template ...
  0.00s Setting up search space for 4 objects ...
  0.12s Constraining search space with 4 relations ...
  0.13s Setting up retrieval plan ...
  0.15s Ready to deliver results from 292337 nodes
Iterate over S.fetch() to get the results
See S.showPlan() to interpret the results
  0.15s The results are connected to the original search template as follows:
 0     
 1 R0  verse
 2 R1      clause
 3 R2          clause_atom
 4 R3          <: clause_atom
 5     
  0.00s Counting results per 1000 up to  the end of the results ...
  0.84s Done: 0 results


Conclusion: there is always textual material between clause_atoms of the same clause.
If we lift the adjacency to sequentially before (`<<`) we do get results:

In [42]:
query = '''
verse
    clause
        clause_atom
        << clause_atom
'''
S.study(query)
S.showPlan()
S.count(progress=1000, limit=-1)
for r in S.fetch(amount=10):
    print(S.glean(r))

  0.00s Checking search template ...
  0.00s Setting up search space for 4 objects ...
  0.11s Constraining search space with 4 relations ...
  0.12s Setting up retrieval plan ...
  0.14s Ready to deliver results from 292337 nodes
Iterate over S.fetch() to get the results
See S.showPlan() to interpret the results
  0.15s The results are connected to the original search template as follows:
 0     
 1 R0  verse
 2 R1      clause
 3 R2          clause_atom
 4 R3          << clause_atom
 5     
  0.00s Counting results per 1000 up to  the end of the results ...
   |     0.30s 1000
   |     0.60s 2000
  0.75s Done: 2587 results
Genesis 1:7 clause[וַיַּבְדֵּ֗ל בֵּ֤ין הַמַּ֨יִם֙ ...] clause_atom[וַיַּבְדֵּ֗ל בֵּ֤ין הַמַּ֨יִם֙ ] clause_atom[וּבֵ֣ין הַמַּ֔יִם ]
Genesis 1:11 clause[תַּֽדְשֵׁ֤א הָאָ֨רֶץ֙ דֶּ֔שֶׁא עֵ֚שֶׂב ...] clause_atom[תַּֽדְשֵׁ֤א הָאָ֨רֶץ֙ דֶּ֔שֶׁא עֵ֚שֶׂב ] clause_atom[עֵ֣ץ פְּרִ֞י ]
Genesis 1:11 clause[עֹ֤שֶׂה פְּרִי֙ לְמִינֹ֔ו עַל־...] clause_atom[עֹ֤שֶׂה פְּרִי֙ לְמִינֹ֔ו

# Nearness for := =: :: :> <:

The relations with `:` in their name always have a requirement somewhere that a slot of the
left hand node equals a slot of the right hand node, or that the two are adjacent.

All these relationships can be relaxed by a **nearness number**.
If you put a number *k* inside the relationship symbols, those restrictions will be relaxed to
*the one slot and the other slot should have a mutual distance of at most k*.

Here is an example.

First we look for clauses, with a phrase in it that starts at the
same slot as the clause.

In [44]:
S.study('''
chapter book=Genesis chapter=1
    clause
      =: phrase
''')
S.count(progress=100, limit=-1)

  0.00s Checking search template ...
  0.00s Setting up search space for 3 objects ...
  0.13s Constraining search space with 3 relations ...
  0.24s Setting up retrieval plan ...
  0.24s Ready to deliver results from 259 nodes
Iterate over S.fetch() to get the results
See S.showPlan() to interpret the results
  0.00s Counting results per 100 up to  the end of the results ...
   |     0.00s 100
  0.00s Done: 129 results


Now we add a bit of freedom, but not much: 0. Indeed, this is no extra
freedom, and it should give the same number of results.

In [45]:
S.study('''
chapter book=Genesis chapter=1
    clause
      =0: phrase
''')
S.count(progress=100, limit=-1)

  0.00s Checking search template ...
  0.00s Setting up search space for 3 objects ...
  0.09s Constraining search space with 3 relations ...
  0.24s Setting up retrieval plan ...
  0.24s Ready to deliver results from 259 nodes
Iterate over S.fetch() to get the results
See S.showPlan() to interpret the results
  0.00s Counting results per 100 up to  the end of the results ...
   |     0.00s 100
  0.00s Done: 129 results


Now we add real freedom: 1 and 2

In [46]:
S.study('''
chapter book=Genesis chapter=1
    clause
      =1: phrase
''')
S.count(progress=100, limit=-1)

  0.00s Checking search template ...
  0.00s Setting up search space for 3 objects ...
  0.10s Constraining search space with 3 relations ...
  0.25s Setting up retrieval plan ...
  0.25s Ready to deliver results from 415 nodes
Iterate over S.fetch() to get the results
See S.showPlan() to interpret the results
  0.00s Counting results per 100 up to  the end of the results ...
   |     0.04s 100
   |     0.04s 200
  0.05s Done: 241 results


In [49]:
S.study('''
chapter book=Genesis chapter=1
    clause
      =2: phrase
''')
S.count(progress=100, limit=-1)

  0.00s Checking search template ...
  0.00s Setting up search space for 3 objects ...
  0.11s Constraining search space with 3 relations ...
  0.22s Setting up retrieval plan ...
  0.22s Ready to deliver results from 479 nodes
Iterate over S.fetch() to get the results
See S.showPlan() to interpret the results
  0.00s Counting results per 100 up to  the end of the results ...
   |     0.00s 100
   |     0.01s 200
   |     0.01s 300
  0.01s Done: 320 results


Let us see some cases:

In [50]:
for r in S.fetch(amount=10):
    print(S.glean(r))

 clause[בְּרֵאשִׁ֖ית בָּרָ֣א אֱלֹהִ֑ים אֵ֥ת ...] phrase[בְּרֵאשִׁ֖ית ]
 clause[בְּרֵאשִׁ֖ית בָּרָ֣א אֱלֹהִ֑ים אֵ֥ת ...] phrase[בָּרָ֣א ]
 clause[וְהָאָ֗רֶץ הָיְתָ֥ה תֹ֨הוּ֙ ...] phrase[וְ]
 clause[וְהָאָ֗רֶץ הָיְתָ֥ה תֹ֨הוּ֙ ...] phrase[הָאָ֗רֶץ ]
 clause[וְחֹ֖שֶׁךְ עַל־פְּנֵ֣י תְהֹ֑ום ] phrase[וְ]
 clause[וְחֹ֖שֶׁךְ עַל־פְּנֵ֣י תְהֹ֑ום ] phrase[חֹ֖שֶׁךְ ]
 clause[וְחֹ֖שֶׁךְ עַל־פְּנֵ֣י תְהֹ֑ום ] phrase[עַל־פְּנֵ֣י תְהֹ֑ום ]
 clause[וְר֣וּחַ אֱלֹהִ֔ים מְרַחֶ֖פֶת עַל־...] phrase[וְ]
 clause[וְר֣וּחַ אֱלֹהִ֔ים מְרַחֶ֖פֶת עַל־...] phrase[ר֣וּחַ אֱלֹהִ֔ים ]
 clause[וַיֹּ֥אמֶר אֱלֹהִ֖ים ] phrase[יֹּ֥אמֶר ]


The first and second result show the same clause, with its first and second phrase respectively.

Note that we look for phrases that lie embedded in their clause.
So we do not get phrases of a preceding clause.

But if we want, we can get those as well.

In [51]:
S.study('''
chapter book=Genesis chapter=1
    c:clause
    p:phrase
    
    c =2: p
''')
S.count(progress=100, limit=-1)

  0.00s Checking search template ...
  0.00s Setting up search space for 3 objects ...
  0.09s Constraining search space with 3 relations ...
  0.48s Setting up retrieval plan ...
  0.48s Ready to deliver results from 479 nodes
Iterate over S.fetch() to get the results
See S.showPlan() to interpret the results
  0.00s Counting results per 100 up to  the end of the results ...
   |     0.00s 100
   |     0.00s 200
   |     0.00s 300
   |     0.01s 400
  0.01s Done: 490 results


We have more results now. Here is a closer look:

In [55]:
S.study('''
verse book=Genesis chapter=1 verse=3
    c:clause
    p:phrase
    
    c =2: p
''')

S.count(progress=100, limit=-1)
for r in S.fetch(amount=100):
    print(S.glean(r))

  0.00s Checking search template ...
  0.00s Setting up search space for 3 objects ...
  0.16s Constraining search space with 3 relations ...
  0.50s Setting up retrieval plan ...
  0.50s Ready to deliver results from 12 nodes
Iterate over S.fetch() to get the results
See S.showPlan() to interpret the results
  0.00s Counting results per 100 up to  the end of the results ...
  0.00s Done: 13 results
Genesis 1:3 clause[וַיֹּ֥אמֶר אֱלֹהִ֖ים ] phrase[יֹּ֥אמֶר ]
Genesis 1:3 clause[וַיֹּ֥אמֶר אֱלֹהִ֖ים ] phrase[אֱלֹהִ֖ים ]
Genesis 1:3 clause[וַיֹּ֥אמֶר אֱלֹהִ֖ים ] phrase[וַ]
Genesis 1:3 clause[יְהִ֣י אֹ֑ור ] phrase[יֹּ֥אמֶר ]
Genesis 1:3 clause[יְהִ֣י אֹ֑ור ] phrase[אֱלֹהִ֖ים ]
Genesis 1:3 clause[יְהִ֣י אֹ֑ור ] phrase[יְהִ֣י ]
Genesis 1:3 clause[יְהִ֣י אֹ֑ור ] phrase[אֹ֑ור ]
Genesis 1:3 clause[יְהִ֣י אֹ֑ור ] phrase[וַֽ]
Genesis 1:3 clause[וַֽיְהִי־אֹֽור׃ ] phrase[יְהִ֣י ]
Genesis 1:3 clause[וַֽיְהִי־אֹֽור׃ ] phrase[אֹ֑ור ]
Genesis 1:3 clause[וַֽיְהִי־אֹֽור׃ ] phrase[וַֽ]
Genesis 1:3 clause[

Here you see in result 4 a phrase of the previous clause in the result.